Data cleaning order:


1. Clean out null from Budget   DONE
2. Clean out null from Revenue  DONE
3. Genre  DONE
4. Clean out null from runtime DONE
5. Genre  REPEATED/DONE
6. Number of competitors (movie releases within a month and same year) DONE
7. First actor DONE
7. Actors previous average profit  DONE
8. Clean out nulls from actor DONE
9. Delete credits DONE
10. Clean out nulls from release date
11. Month DONE
12. Season DONE
13. pg rating + add unrated category
14. Director
15. Directors previous average profit


# **Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from datetime import datetime
import math

#this will mute errors that made your final pdf less legible
import warnings
warnings.filterwarnings('ignore')

# **Loading drive**

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Loading data**

In [ ]:
# load the data
#df = pd.read_csv('/content/drive/MyDrive/BA476GroupProject/MOVIES/movies.csv')
df = pd.read_csv('/content/drive/MyDrive/MOVIES/movies.csv')

#testt

df.shape

(722473, 20)

# **Cleaning data**

In [ ]:
# Dropping 'poster_path','backdrop_path', 'status', 'recommendations' columns
df_cleaned = df.drop(['poster_path','popularity','overview', 'backdrop_path','status','recommendations','tagline','vote_average','vote_count','keywords'], axis=1)

# Displaying the cleaned DataFrame
df_cleaned.shape

(722473, 10)

In [ ]:
hasBudget=df_cleaned[df_cleaned['budget']>=10000]
hasRev=hasBudget[hasBudget['revenue']!=0]
hasRev['genres'].isnull().sum()

145

In [ ]:
genresCleaned=hasRev[hasRev['genres'].notnull()]
split_genres = genresCleaned['genres'].str.split('-', expand=True)
genre_columns = [f'Genre{i}' for i in range(1, 8)]
split_genres.columns = genre_columns
columns_to_drop = [f'Genre{i}' for i in range(4, 8)]
split_genres.drop(columns=columns_to_drop, inplace=True)
split_genres.fillna('', inplace=True)
all_genres = split_genres['Genre1'] + ',' + split_genres['Genre2'] + ',' + split_genres['Genre3']
split_genres['Genre1'].value_counts()

Genre1
Drama              2313
Comedy             2041
Action             1435
Horror              628
Adventure           570
Crime               443
Thriller            393
Animation           302
Romance             297
Science Fiction     221
Fantasy             210
Family              189
Documentary         130
Mystery             115
Music               106
War                  92
Western              67
History              62
TV Movie              2
Name: count, dtype: int64

In [ ]:

# Split concatenated genres into individual genres
all_genres = all_genres.str.split(',')

# Flatten the list of genres
all_genres = [genre for sublist in all_genres for genre in sublist]

# Count occurrences of each genre
genre_counts = pd.Series(all_genres).value_counts()

# Select the 10 most common genres
top_10_genres = genre_counts.head(20).index

# Create dummy variables for the 10 most common genres
for genre in top_10_genres:
    split_genres[genre] = split_genres.apply(lambda row: 1 if genre in row.values else 0, axis=1)

# Drop the original genre columns
split_genres.drop(columns=['Genre1', 'Genre2', 'Genre3'], inplace=True)

split_genres.drop(columns=[''], inplace=True)

In [ ]:
WithGenres = pd.concat([genresCleaned, split_genres], axis=1)
###Dummy Variables for Genre Added here

In [ ]:
RuntimeCleaned = WithGenres.dropna(subset=['runtime'])
RuntimeCleaned.shape
##Null runtime values dropped


(9609, 29)

In [ ]:
withFirstActor = RuntimeCleaned
withFirstActor['firstActor'] = withFirstActor['credits'].str.split('-').str[0]
##get first actor


In [ ]:
withFirstActor.dropna(subset=['release_date'], inplace=True)
##drop null releases

withFirstActor[['year', 'month', 'day']] = withFirstActor['release_date'].str.split('-', expand=True)
##add in new variables with string values

withFirstActor['year'] = withFirstActor['year'].astype(int)
withFirstActor['month'] = withFirstActor['month'].astype(int)
withFirstActor['day'] = withFirstActor['day'].astype(int)
# Convert the new columns to integers for easier comparison

# Drop the original 'release_date' column if you don't need it anymore


In [ ]:
budgetAdded = withFirstActor
budgetAdded['revenue_to_budget_ratio'] = budgetAdded['revenue'] / budgetAdded['budget']
##add in Y variable
budgetAdded.dropna(subset=['firstActor'], inplace=True)
##drop null actors

In [ ]:
firstActorCalculation = budgetAdded

# Convert 'year', 'month', and 'day' to datetime format
firstActorCalculation['release_date'] = pd.to_datetime(firstActorCalculation[['year', 'month', 'day']])

# Sort the DataFrame by release date
firstActorCalculation = firstActorCalculation.sort_values(by=['firstActor', 'release_date'])

# Calculate 'actor_previous_success' for each actor
firstActorCalculation['actor_previous_success'] = firstActorCalculation.groupby('firstActor')['revenue_to_budget_ratio'].transform(lambda x: x.expanding().mean().shift())

# Fill missing values with 0 for actors with no prior releases
firstActorCalculation['actor_previous_success'] = firstActorCalculation.groupby('firstActor')['actor_previous_success'].transform(lambda x: x.fillna(0))

# Drop intermediate columns if needed
firstActorCalculation.drop(columns=['release_date'], inplace=True)

firstActorCalculation.head()

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,War,Documentary,Western,TV Movie,firstActor,year,month,day,revenue_to_budget_ratio,actor_previous_success
8436,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,0,0,0,0,50 Cent,2005,11,9,1.161063,0.000000
15022,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,0,0,0,0,50 Cent,2011,10,17,0.106409,1.161063
2328,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,0,0,0,0,A.J. Cook,2003,1,31,3.497736,0.000000
184848,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0,0,0,0,Aamina Sheikh,2018,3,30,2.428571,0.000000
231976,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0,0,0,0,Aamina Sheikh,2018,3,30,2.428571,2.428571


In [ ]:
movies_df = firstActorCalculation

# drop rows with 'null' in year
movies_df.dropna(subset=['year'])

# get the day of the week from the year, month, and day
def get_day_of_week(year, month, day):
  day_of_week = datetime(year, month, day).weekday()
  return day_of_week + 1 # sun= 1, mon= 2...

# get the season
def get_season_numeric(month):
  if month in [9, 10, 11]:
    return 1 # fall
  elif month in [12, 1, 2]:
    return 2 # winter
  elif month in [3, 4, 5]:
    return 3 # spring
  elif month in [6, 7, 8]:
    return 4 # summer

# Apply the numeric functions to the dataframe
movies_df['day_of_week_numeric'] = movies_df.apply(lambda x: get_day_of_week(x['year'], x['month'], x['day']), axis=1)
movies_df['season_numeric'] = movies_df['month'].apply(get_season_numeric)
movies_df.head()


,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,Western,TV Movie,firstActor,year,month,day,revenue_to_budget_ratio,actor_previous_success,day_of_week_numeric,season_numeric
8436,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,0,0,50 Cent,2005,11,9,1.161063,0.000000,3,1
15022,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,0,0,50 Cent,2011,10,17,0.106409,1.161063,1,1
2328,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,0,0,A.J. Cook,2003,1,31,3.497736,0.000000,5,2
184848,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0,0,Aamina Sheikh,2018,3,30,2.428571,0.000000,5,3
231976,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0,0,Aamina Sheikh,2018,3,30,2.428571,2.428571,5,3


In [ ]:
# Calculate the number of competitors released within the same month and year
competitors_count = movies_df.groupby(['year', 'month']).size() - 1
movies_df['competitors'] = movies_df.apply(lambda x: competitors_count.loc[(x['year'], x['month'])], axis=1)
movies_df.head()

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,TV Movie,firstActor,year,month,day,revenue_to_budget_ratio,actor_previous_success,day_of_week_numeric,season_numeric,competitors
8436,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,0,50 Cent,2005,11,9,1.161063,0.000000,3,1,14
15022,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,0,50 Cent,2011,10,17,0.106409,1.161063,1,1,28
2328,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,0,A.J. Cook,2003,1,31,3.497736,0.000000,5,2,15
184848,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0,Aamina Sheikh,2018,3,30,2.428571,0.000000,5,3,29
231976,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0,Aamina Sheikh,2018,3,30,2.428571,2.428571,5,3,29


**Data fetch for directors from TMDB API**

In [ ]:
!pip install tmdbsimple

In [ ]:
df=movies_df
df.shape


(9545, 37)

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY = '4af841124f67da599980a864dedc795f'
print(dir(tmdb))

['APIKeyError', 'API_KEY', 'API_VERSION', 'Account', 'Authentication', 'Certifications', 'Changes', 'Collections', 'Companies', 'Configuration', 'Credits', 'Discover', 'Find', 'Genres', 'GuestSessions', 'Keywords', 'Lists', 'Movies', 'Networks', 'People', 'REQUESTS_SESSION', 'REQUESTS_TIMEOUT', 'Reviews', 'Search', 'TV', 'TV_Changes', 'TV_Episode_Groups', 'TV_Episodes', 'TV_Seasons', 'Trending', '__all__', '__author__', '__builtins__', '__cached__', '__copyright__', '__doc__', '__file__', '__license__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__title__', '__version__', 'account', 'base', 'changes', 'configuration', 'discover', 'find', 'genres', 'movies', 'os', 'people', 'requests', 'search', 'tv']


In [ ]:
import concurrent.futures

# Define the get_director function
def get_director(movie_id):
    try:
        movie = tmdb.Movies(movie_id)
        credits = movie.credits()

        # Iterate through the crew
        for crew_member in credits['crew']:
            if crew_member['job'] == 'Director':
                return crew_member['name']
        return None

    except Exception as e:
        print(f"Error occurred for movie ID {movie_id}: {e}")
        return None

# Function to process a batch of movie IDs
def process_batch(movie_ids):
    directors = {}
    for movie_id in movie_ids:
        directors[movie_id] = get_director(movie_id)
    return directors

# Adding director column to data
# Create a new column 'director'
df['director'] = ''

# Split movie IDs into batches for processing
batch_size = 100  # You can adjust the batch size as needed
movie_id_batches = [df['id'][i:i+batch_size].tolist() for i in range(0, len(df), batch_size)]

# Process batches using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_batch, movie_id_batches))

# Update the 'director' column in the DataFrame
for result in results:
    for movie_id, director_name in result.items():
        df.loc[df['id'] == movie_id, 'director'] = director_name

# Display the updated DataFrame
df.head()

Error occurred for movie ID 825721: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/825721/credits?api_key=4af841124f67da599980a864dedc795f
Error occurred for movie ID 825721: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/825721/credits?api_key=4af841124f67da599980a864dedc795f
Error occurred for movie ID 944450: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/944450/credits?api_key=4af841124f67da599980a864dedc795f
Error occurred for movie ID 944451: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/944451/credits?api_key=4af841124f67da599980a864dedc795f
Error occurred for movie ID 944447: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/944447/credits?api_key=4af841124f67da599980a864dedc795f
Error occurred for movie ID 788134: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/788134/credits?api_key=4af841124f67da599980a864dedc795f
Error occurred f

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,firstActor,year,month,day,revenue_to_budget_ratio,actor_previous_success,day_of_week_numeric,season_numeric,competitors,director
8436,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,50 Cent,2005,11,9,1.161063,0.000000,3,1,14,Jim Sheridan
15022,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,50 Cent,2011,10,17,0.106409,1.161063,1,1,28,Mike Gunther
2328,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,A.J. Cook,2003,1,31,3.497736,0.000000,5,2,15,David R. Ellis
184848,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,Aamina Sheikh,2018,3,30,2.428571,0.000000,5,3,29,Asim Abbasi
231976,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,Aamina Sheikh,2018,3,30,2.428571,2.428571,5,3,29,Asim Abbasi


In [ ]:
#Drop rows where 'director' is null
df.dropna(subset=['director'], inplace=True)
df.shape

(9489, 39)

In [ ]:
# API Fetch get AGE RATING
#Define the get_age_rating function
def get_age_rating(movie_id):
    try:
        # Retrieve information about the movie
        movie = tmdb.Movies(movie_id)
        release_dates = movie.release_dates()

        # Iterate through the release dates to find the age rating
        for country_data in release_dates['results']:
            if country_data['iso_3166_1'] == 'US':  # Check for the primary country (e.g., US)
                for release_date in country_data['release_dates']:
                    if 'certification' in release_date:
                        return release_date['certification']
                break

        return None  # Return None if age rating is not available

    except Exception as e:
        print(f"Error occurred for movie ID {movie_id}: {e}")
        return None

# Function to process a batch of movie IDs and fetch age ratings
def process_batch_age_ratings(movie_ids):
    age_ratings = {}
    for movie_id in movie_ids:
        age_ratings[movie_id] = get_age_rating(movie_id)
    return age_ratings


# Get all movie IDs from the DataFrame
movie_ids = df['id'].tolist()

# Split movie IDs into batches for processing
batch_size = 100  # You can adjust the batch size as needed
movie_id_batches = [movie_ids[i:i+batch_size] for i in range(0, len(movie_ids), batch_size)]

# Adding age_rating column to data
# Create a new column 'age_rating'
df['age_rating'] = ''

# Process batches using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    age_rating_results = list(executor.map(process_batch_age_ratings, movie_id_batches))

# Update the 'age_rating' column in the DataFrame
for result in age_rating_results:
    for movie_id, age_rating in result.items():
        df.loc[df['id'] == movie_id, 'age_rating'] = age_rating

# Display the updated DataFrame
df.head()

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,year,month,day,revenue_to_budget_ratio,actor_previous_success,day_of_week_numeric,season_numeric,competitors,director,age_rating
8436,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,2005,11,9,1.161063,0.000000,3,1,14,Jim Sheridan,R
15022,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,2011,10,17,0.106409,1.161063,1,1,28,Mike Gunther,R
2328,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,2003,1,31,3.497736,0.000000,5,2,15,David R. Ellis,
184848,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,2018,3,30,2.428571,0.000000,5,3,29,Asim Abbasi,None
231976,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,2018,3,30,2.428571,2.428571,5,3,29,Asim Abbasi,None


In [ ]:
# Replace null values in 'age_rating' with 'unrated'
df['age_rating'].fillna('unrated', inplace=True)

# Replace empty strings ('') in 'age_rating' with 'unrated'
df['age_rating'].replace('', 'unrated', inplace=True)
df.shape
df.head()

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,actor_previous_success,day_of_week_numeric,season_numeric,competitors,director,age_rating,genre_competition_x,genre_competition_y,genre_competition_original,genre_competition_genre
0,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,0.000000,3,1,14,Jim Sheridan,R,1,1,1,1
1,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,1.161063,1,1,28,Mike Gunther,R,2,2,2,2
2,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,0.000000,5,2,15,David R. Ellis,unrated,2,2,2,2
3,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,0.000000,5,3,29,Asim Abbasi,unrated,2,2,2,2
4,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,2.428571,5,3,29,Asim Abbasi,unrated,2,2,2,2


In [ ]:
# Group by year and genre, then count the number of releases for each genre within a year
genre_specific_competition = df.groupby(['year', 'genres']).size().reset_index(name='genre_competition')

# Merge the genre competition count information back into the original DataFrame
df = pd.merge(df, genre_specific_competition, how='left', on=['year', 'genres'], suffixes=('_original', '_genre'))

# Fill NaN values in case a particular genre has no competition
df['genre_competition'].fillna(0, inplace=True)

df.head(20)

#if the 'genre_competition' value for this entry is 1, it means that within the year 2023,
#there were 1 other movies released that belong to the same combination of genres

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,month,day,revenue_to_budget_ratio,actor_previous_success,day_of_week_numeric,season_numeric,competitors,director,age_rating,genre_competition
0,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,11,9,1.161063,0.000000,3,1,14,Jim Sheridan,R,1
1,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,10,17,0.106409,1.161063,1,1,28,Mike Gunther,R,2
2,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,1,31,3.497736,0.000000,5,2,15,David R. Ellis,unrated,2
3,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,3,30,2.428571,0.000000,5,3,29,Asim Abbasi,unrated,2
4,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,3,30,2.428571,2.428571,5,3,29,Asim Abbasi,unrated,2
5,21570,Rangeela,Drama-Romance,hi,Jhamu Sughand Productions-Varma Creations,2800000.0,14000000.0,175.0,Aamir Khan-Urmila Matondkar-Jackie Shroff-Guls...,1,...,9,8,5.000000,0.000000,5,1,15,Ram Gopal Varma,unrated,9
6,31524,Sarfarosh,Action-Drama,hi,Cinematt Pictures Pvt Ltd,8500000.0,9100000.0,174.0,Aamir Khan-Sonali Bendre-Naseeruddin Shah-Muke...,1,...,4,30,1.070588,5.000000,5,3,17,John Mathew Matthan,unrated,1
7,19666,Lagaan: Once Upon a Time in India,Adventure-Romance-Drama,hi,Aamir Khan Productions-Ashutosh Gowariker Prod...,5200000.0,8100000.0,224.0,Aamir Khan-Gracy Singh-Rachel Shelley-Paul Bla...,1,...,6,15,1.557692,3.035294,5,4,16,Ashutosh Gowariker,PG,1
8,14752,Dil Chahta Hai,Drama-Romance,hi,Excel Entertainment,2079000.0,4099000.0,183.0,Aamir Khan-Saif Ali Khan-Akshaye Khanna-Preity...,1,...,7,24,1.971621,2.542760,2,4,17,Farhan Akhtar,unrated,12
9,20496,Mangal Pandey - The Rising,Drama,hi,NaN,70000000.0,7250000.0,150.0,Aamir Khan-Rani Mukerji-Toby Stephens-Ameesha ...,1,...,8,12,0.103571,2.399975,5,4,20,Ketan Mehta,unrated,23


In [ ]:
df.shape

(9489, 40)

In [ ]:
# Group by 'director' and calculate average 'revenue_to_budget_ratio' for movies released before each movie
director_previous_success = df.groupby('director').apply(lambda group: group[group['year'] < group['year'].iloc[0]]['revenue_to_budget_ratio'].mean())

# Rename the resulting Series to facilitate the merge
director_previous_success.rename('director_previous_success', inplace=True)

# Merge the calculated averages back into the original DataFrame
df = df.merge(director_previous_success, how='left', left_on='director', right_index=True)

# Fill missing values with zero
df['director_previous_success'].fillna(0, inplace=True)

df.head()

,id,title,genres,original_language,production_companies,budget,revenue,runtime,credits,Drama,...,day,revenue_to_budget_ratio,actor_previous_success,day_of_week_numeric,season_numeric,competitors,director,age_rating,genre_competition,director_previous_success
0,10060,Get Rich or Die Tryin',Drama-Crime-Music,en,Cent Productions Inc.-Paramount-MTV Films,40000000.0,46442528.0,117.0,50 Cent-Joy Bryant-Adewale Akinnuoye-Agbaje-Om...,1,...,9,1.161063,0.000000,3,1,14,Jim Sheridan,R,1,13.956445
1,70586,Setup,Action-Crime-Drama-Thriller,en,Grindstone Entertainment Group-Cheetah Vision-...,20000000.0,2128186.0,85.0,50 Cent-Ryan Phillippe-Bruce Willis-James Rema...,1,...,17,0.106409,1.161063,1,1,28,Mike Gunther,R,2,0.000000
2,9358,Final Destination 2,Horror-Mystery,en,New Line Cinema-Zide-Perry Productions,26000000.0,90941129.0,90.0,A.J. Cook-Ali Larter-Michael Landes-Terrence '...,0,...,31,3.497736,0.000000,5,2,15,David R. Ellis,unrated,2,0.000000
3,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,30,2.428571,0.000000,5,3,29,Asim Abbasi,unrated,2,0.000000
4,512096,Cake,Drama-Romance-Mystery-Comedy,ur,Indus Takies-ZAB Films,350000.0,850000.0,125.0,Aamina Sheikh-Sanam Saeed-Adnan Malik-Faris Kh...,1,...,30,2.428571,2.428571,5,3,29,Asim Abbasi,unrated,2,0.000000


In [ ]:
df.shape

(9489, 41)

In [ ]:
df.to_csv('/content/drive/MyDrive/MOVIES/AllClean.csv', index=False)